In [ ]:
import pandas as pd
import numpy as np
import folium
from matplotlib import pyplot as plt
from io import BytesIO
import base64

intersection_data = pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcIntersection.CSV')
count_data = pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCount.CSV')
count_data['StartDate'] = (count_data['StartDate'] / 1e9).astype(int).astype(str)
count_data['StartDate'] = pd.to_datetime(count_data['StartDate'], format='%Y%m%d', errors = 'coerce')
count_data['Year'] = count_data['StartDate'].dt.year
count_data.dropna(subset = ['IntersectionId', 'Year'], inplace = True)
count_data['IntersectionId'] = count_data['IntersectionId'].astype(int)
most_recent_counts = count_data.loc[count_data.groupby('IntersectionId')['StartDate'].idxmax()]

def calculate_radius(count, max_count):
    return (count / max_count) * 30

am_peak_start = pd.to_datetime('06:30:00', format = '%H:%M:%S').time()
am_peak_end = pd.to_datetime('08:30:00', format = '%H:%M:%S').time()
pm_peak_start = pd.to_datetime('16:30:00', format = '%H:%M:%S').time()
pm_peak_end = pd.to_datetime('18:30:00', format = '%H:%M:%S').time()

chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(most_recent_counts, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 
                                 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 
                                 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 1]
    chunks.append(chunk)
cars = pd.concat(chunks, ignore_index = True)
car_summary = cars.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 
                            'Longitude', 'StartDate'])['TotalCount'].sum().reset_index()
max_count = car_summary['TotalCount'].max()
m1 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in car_summary.iterrows():
    if row['TotalCount'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['TotalCount'], max_count),
            color = '#89d1ff',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId']}<br>" + 
            f"<b>Date:</b> {row['StartDate'].strftime('%Y-%m-%d')}<br>" + f"<b>Total Count:</b> {row['TotalCount']}"
        ).add_to(m1)

legend_html = '''
<div style = 'position: fixed; 
        bottom: 50px; left: 50px; width: 200px; height: auto; 
        border:2px solid grey; z-index:9999; font-size:14px;
        background-color:white; padding: 10px;'>
<b>Car Total Count</b><br>
'''
for count in [5000, 10000, 15000, 20000]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; 
        background-color: #89d1ff; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m1.get_root().html.add_child(folium.Element(legend_html))
m1.save('car.html')
print('Map has been created and saved to car.html')

Map has been created and saved to car.html


In [ ]:
chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(most_recent_counts, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 'ThruOrPedMovement', 
                                 'BearRightMovement', 'RightMovement', 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 2]
    chunks.append(chunk)

trucks = pd.concat(chunks, ignore_index = True)
truck_summary = trucks.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 'Longitude', 'StartDate'])['TotalCount'].sum().reset_index()
max_count = truck_summary['TotalCount'].max()
m2 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in truck_summary.iterrows():
    if row['TotalCount'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['TotalCount'], max_count),
            color = '#9a9a9a',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId']}<br>" + f"<b>Date:</b> {row['StartDate'].strftime('%Y-%m-%d')}<br>" + f"<b>Total Count:</b> {row['TotalCount']}"
        ).add_to(m2)

legend_html = '''
<div style = 'position: fixed; 
        bottom: 50px; left: 50px; width: 200px; height: auto; 
        border:2px solid grey; z-index:9999; font-size:14px;
        background-color:white; padding: 10px;'>
<b>Truck Total Count</b><br>
'''
for count in [10, 50, 100, 150]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; background-color: #9a9a9a; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m2.get_root().html.add_child(folium.Element(legend_html))
m2.save('truck.html')
print('Map has been created and saved to truck.html')

124

In [ ]:
chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(most_recent_counts, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 3]
    chunks.append(chunk)

bikes = pd.concat(chunks, ignore_index = True)
bike_summary = bikes.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 'Longitude', 'StartDate'])['TotalCount'].sum().reset_index()
max_count = bike_summary['TotalCount'].max()
m3 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in bike_summary.iterrows():
    if row['TotalCount'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['TotalCount'], max_count),
            color = '#ff8b8b',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId']}<br>" + f"<b>Date:</b> {row['StartDate'].strftime('%Y-%m-%d')}<br>" + f"<b>Total Count:</b> {row['TotalCount']}"
        ).add_to(m3)

legend_html = '''
<div style = 'position: fixed; 
        bottom: 50px; left: 50px; width: 200px; height: auto; 
        border:2px solid grey; z-index:9999; font-size:14px;
        background-color:white; padding: 10px;'>
<b>Bike Total Count</b><br>
'''
for count in [100, 200, 300, 400]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; background-color: #ff8b8b; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m3.get_root().html.add_child(folium.Element(legend_html))
m3.save('bike.html')
print('Map has been created and saved to bike.html')

374

In [ ]:
chunks = []
for chunk in pd.read_csv('D:/Dr. Shamshiripour/PAG_TMC/TmcCountData.CSV', chunksize = 50000):
    chunk['StartTime'] = pd.to_datetime(chunk['StartTime'], format = '%H:%M:%S').dt.time
    chunk = chunk[((chunk['StartTime'] >= am_peak_start) & (chunk['StartTime'] < am_peak_end)) | 
                  ((chunk['StartTime'] >= pm_peak_start) & (chunk['StartTime'] < pm_peak_end))]
    chunk = chunk.merge(most_recent_counts, on = 'CountId', how = 'inner')
    chunk = chunk.merge(intersection_data, on = 'IntersectionId', how = 'inner')
    chunk['TotalCount'] = chunk[['HardLeftMovement', 'LeftMovement', 'BearLeftMovement', 'ThruOrPedMovement', 'BearRightMovement', 'RightMovement', 'HardRightMovement', 'UTurnMovement']].sum(axis = 1)
    chunk = chunk[chunk['ClassificationId'] == 4]
    chunks.append(chunk)

pedestrians = pd.concat(chunks, ignore_index = True)
pedestrian_summary = pedestrians.groupby(['IntersectionId', 'ClassificationId', 'Latitude', 'Longitude', 'StartDate'])['TotalCount'].sum().reset_index()
max_count = pedestrian_summary['TotalCount'].max()
m4 = folium.Map(location = [32.2226, -110.9747], zoom_start = 10, tiles = 'cartodbpositron')
for index, row in pedestrian_summary.iterrows():
    if row['TotalCount'] > 0:
        folium.CircleMarker(
            location = (row['Latitude'], row['Longitude']),
            radius = calculate_radius(row['TotalCount'], max_count),
            color = '#a7ff94',
            fill = True,
            fill_opacity = 0.5,
            popup = f"<b>Intersection ID:</b> {row['IntersectionId']}<br>" + f"<b>Date:</b> {row['StartDate'].strftime('%Y-%m-%d')}<br>" + f"<b>Total Count:</b> {row['TotalCount']}"
        ).add_to(m4)

legend_html = '''
<div style = 'position: fixed; 
        bottom: 50px; left: 50px; width: 200px; height: auto; 
        border:2px solid grey; z-index:9999; font-size:14px;
        background-color:white; padding: 10px;'>
<b>Pedestrian Total Count</b><br>
'''
for count in [500, 1000, 1500, 2000]:
    radius = calculate_radius(count, max_count) * 1.5
    legend_html += f'''
    <div style = 'margin: 5px 0;'>
        {count}
        <div style = 'display: inline-block; width: {radius:.2f}px; height: {radius:.2f}px; background-color: #a7ff94; border-radius: 100%; margin-left: 10px;'></div>
    </div>
    '''
legend_html += '</div>'
m4.get_root().html.add_child(folium.Element(legend_html))
m4.save('pedestrian.html')
print('Map has been created and saved to pedestrian.html')

1673